In [9]:
import json
import os
import glob
from config import problem_configs, provider_configs

# Configuration
SOURCE_DIR = "individual_results"
FINAL_OUTPUT_DIR = "isingBatches"

In [10]:
def merge_all_problem_classes():
    """
    Finds and merges individual result files for each problem class
    defined in the problem_configs dictionary.
    """
    # Loop through each problem type defined in the config
    for problem_name, problem_config in problem_configs.items():
        for provider_name, provider_config in provider_configs.items():

            print(f"--- Processing {problem_name} results from {provider_name} provider ---")

            problem_file_slug = problem_config['file_slug']
            provider_file_slug = provider_config['file_slug']
            print(f"File slugs for {problem_name} on {provider_name}: \"{problem_file_slug}\" and \"{provider_file_slug}\"")

            search_pattern = os.path.join(SOURCE_DIR, f"{problem_file_slug}{provider_file_slug}_num_*.json")
            individual_files = glob.glob(search_pattern)

            if not individual_files:
                print(f"No result files found for pattern: {search_pattern}. Skipping.\n")
                continue  # Move to the next problem class
        
            print(f"Found {len(individual_files)} result files to merge.")

            all_results = []
            metadata = {}

            # Loop through each found file, load its data, and append the result
            for file_path in individual_files:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                
                all_results.append(data["result"])
                
                if not metadata:
                    metadata = data["metadata"]

            all_results.sort(key=lambda x: x['instance_id'])

            # Assemble the final data structure for this problem class
            final_data = {
                "metadata": metadata,
                "results": all_results
            }

            # 2. Dynamically create the final output filename
            final_output_filename = f"MERGED_QAOA_results_{provider_file_slug}_{problem_file_slug}.json"
            final_path = os.path.join(FINAL_OUTPUT_DIR, final_output_filename)

            os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)

            with open(final_path, 'w') as f:
                json.dump(final_data, f, indent=4)

            print(f"Successfully merged {len(all_results)} results into {final_path}\n")


if __name__ == "__main__":
    merge_all_problem_classes()

--- Processing TSP results from IBM provider ---
File slugs for TSP on IBM: "TSP_9q_" and "aer_simulator"
No result files found for pattern: individual_results/TSP_9q_aer_simulator_num_*.json. Skipping.

--- Processing TSP results from IONQ provider ---
File slugs for TSP on IONQ: "TSP_9q_" and "ionq_simulator"
Found 100 result files to merge.
Successfully merged 100 results into isingBatches/MERGED_QAOA_results_ionq_simulator_TSP_9q_.json

--- Processing Knapsack results from IBM provider ---
File slugs for Knapsack on IBM: "Knapsack_6_items_9q_" and "aer_simulator"
No result files found for pattern: individual_results/Knapsack_6_items_9q_aer_simulator_num_*.json. Skipping.

--- Processing Knapsack results from IONQ provider ---
File slugs for Knapsack on IONQ: "Knapsack_6_items_9q_" and "ionq_simulator"
Found 100 result files to merge.
Successfully merged 100 results into isingBatches/MERGED_QAOA_results_ionq_simulator_Knapsack_6_items_9q_.json

--- Processing MinimumVertexCover resu